In [14]:
from transformers import pipeline
import torch

In [15]:
generate_text = pipeline(model="databricks/dolly-v2-3b", torch_dtype=torch.bfloat16, trust_remote_code=True, device_map="auto")


In [3]:
class journallingClass:
    def __init__(self, journalType, title, category, dateCreated, mood, journalClass):
        self.journalType = journalType
        self.journalTitle = title
        self.journalCategory = category 
        self.dateCreated = dateCreated
        self.mood = mood
        self.journalClass = journalClass
    
    def getJournalClass(self):
        return self.journalClass
        
        
class journalEntryClass:
    def __init__(self, userEntry, harmIntent = 0, modelResponse = []):
        self.userEntry = userEntry 
        self.harmIntent = harmIntent
        self.modelResponse = modelResponse
#         self.safetyResponse = safetyResponse
    def printEntry(self):
        print(f"entry: {self.userEntry}, harmful intent: {self.harmIntent}, model response: {self.modelResponse}")

        
    def updateHarmIntent(self, harmIntent):
        self.harmIntent = harmIntent
    
    def addModelResponse(self, response):
        self.modelResponse.append(response)
        
class casualJournal(journallingClass):
    
    def __init__(self, entries = []):
        self.entries = []
        
    def createNewEntry(self, userEntry,  harmIntent = 0, modelResponse = []):
        self.entries.append(journalEntryClass(userEntry, harmIntent, modelResponse))
        return self.entries[-1]
        
    def printJournal(self):
        for entry in self.entries:
            entry.printEntry()
    
    
        
    
class catharticJournal(journallingClass):
    def __init__(self, stage = 0, entries = {}):
        self.stage = stage # default stage 0
        self.entries = entries
        
    
    def createStage(self, journalEntry):
        self.entries[f"stage{self.stage}"] = journalEntry
    
    def updateStage(self):
        if self.stage >=5:
            return 
        self.stage += 1
        
    def createNewEntry(self, userEntry,  harmIntent = 0, modelResponse = []):
        self.updateStage()
        self.entries[f"stage{self.stage}"] = journalEntryClass(userEntry, harmIntent, modelResponse)
        return self.entries[f"stage{self.stage}"]
    
    def printJournal(self):
        for stage in self.entries.keys():
            print(stage)
            self.entries[stage].printEntry()


### Note
* what is it is empty 
* new entry

In [23]:
response = {
    # je 1 and 2 are done, je3 is new request 
    "entry": ["journal entry 1", "journal entry 2", "Jonny's cat died today, it makes me feel sad. Can you provide some empathy"],
    "harmfulIntent": [0,0],
    "modelResponse": ["model response", ["mr 2.1", "mr 2.2"]],
    "type": 0, # assume 0 casual,
    "title": "johny's cat died", 
    "userID": 8940823, 
    "date": "14/12/2023", 
    "category": "cat died",
    "mood": "mood"
    # harm intent 
}

In [24]:
def main():
    
    if response["type"] == 0: # get type check from backend
        journalChild = casualJournal()
    else:
        journalChild = catharticJournal()
        
    # pull journalType, title, category, dateCreated from backend
    journal = journallingClass(response["type"], response["title"], response["category"], response["date"], response["mood"], journalChild)
    
    # pull journal entries, loop through and load 
    for entry, harmfulIntent, modelResponse in zip(response["entry"], response["harmfulIntent"], response["modelResponse"]):
        journal.getJournalClass().createNewEntry(entry, harmfulIntent, modelResponse)
    

    print("hi", response["entry"][-1], "done")
    currentEntry = journal.getJournalClass().createNewEntry(response["entry"][-1])
    
    currentEntry.printEntry()
    harmIntent = checkHarmfulIntent(currentEntry)
    
    if harmIntent: 
        modelResponse = buildHarmresponse() 
        currentEntry.addModelResponse(modelResponse)
        return 
    
    modelResponse = generatePrompt(response["type"], currentEntry)
    currentEntry.addModelResponse(modelResponse)
    
    

In [29]:
res[0]

{'generated_text': 'It is hard to understand what Jonny is going through. Perhaps you could send Jonny a virtual hug or take a break from your workout and pour yourself a cold one.'}

"Johnny broke up with me and I feel so sad... I really liked him and we have been together for 3 years now. I didn't see this coming"

In [30]:
response["entry"].append("Johnny broke up with me and I feel so sad... I really liked him and we have been together for 3 years now. I didn't see this coming")

In [39]:
userEntry = response["entry"][-1]
prompt = f"Consider the following scenario: {userEntry}. Provide me with empathy and comfort."
print(f"promt: {prompt}")
res = generate_text("Jonny's cat died today, it makes me feel sad. Can you provide some empathy")

print(f"response: {res[0]}")

promt: Consider the following scenario: Johnny broke up with me and I feel so sad... I really liked him and we have been together for 3 years now. I didn't see this coming. Provide me with empathy and comfort.
response: {'generated_text': "I am so sorry to hear that your cat has died. It's always hard to lose a companion animal, but understand that they have probably spent their entire life in his company, and were likely quite happy and comfortable in that company. My intuition says that he was probably feeling sleepy and may have had a heart attack, but I could be wrong. My advice to you is to take some time to grieve your cat's loss, but also think about what might have been done to him so that you don't make the same mistake next time. Some people find comfort in detailed thinking about causes and possibilities and I recommend finding a form of this with friends or family to help you cope."}


In [31]:
response

{'entry': ['journal entry 1',
  'journal entry 2',
  "Jonny's cat died today, it makes me feel sad. Can you provide some empathy",
  "Johnny broke up with me and I feel so sad... I really liked him and we have been together for 3 years now. I didn't see this coming"],
 'harmfulIntent': [0, 0],
 'modelResponse': ['model response', ['mr 2.1', 'mr 2.2']],
 'type': 0,
 'title': "johny's cat died",
 'userID': 8940823,
 'date': '14/12/2023',
 'category': 'cat died',
 'mood': 'mood'}

In [17]:
res = generate_text("Jonny's cat died today, it makes me feel sad. Can you provide some empathy")

print(res)

[{'generated_text': 'It is hard to understand what Jonny is going through. Perhaps you could send Jonny a virtual hug or take a break from your workout and pour yourself a cold one.'}]


In [21]:
print("hello")

hello


In [26]:
def checkHarmfulIntent(journalEntry):
    # implement later
    journalEntry.updateHarmIntent(0)
    return False 

def buildHarmresponse():
    response = "Reach out to a therapist :)"
    return response

def generateCatharticResponse(journalEntry):
    return "this is a cathartic response"

def generateCasualResponse(journalEntry):
    print(journalEntry.userEntry)
    res = generate_text(journalEntry.userEntry)
    print(res)
    return res[0]

def promptSafetyDetector(journalEntry):
    return True

def generatePrompt(typeJ, journalEntry):
    if typeJ == 1:
        response = generateCatharticResponse(journalEntry)

    else:
        response = generateCasualResponse(journalEntry)
    
    safety_counter = 0
    while(safety_counter<5):
        safe = promptSafetyDetector(journalEntry)
        safety_counter += 1
        if safe:
            break
    return response

In [27]:
main()

hi Jonny's cat died today, it makes me feel sad. Can you provide some empathy done
entry: Jonny's cat died today, it makes me feel sad. Can you provide some empathy, harmful intent: 0, model response: []
Jonny's cat died today, it makes me feel sad. Can you provide some empathy
[{'generated_text': "Yes of course, I'm so sorry to hear that. I imagine the feeling of losing a companion is difficult, and as a fellow pet owner I can sympathize with your loss. Perhaps you can talk to him/her now, or consider getting a support animal (e.g. a dog) to accompany you while you grieve."}]


# Pseudocode

Main Function:
	# create / look for journal entry class (ask SS)
	journal = journalParentClass # backend people will send this
	createNewJournal(userEntry)

	checkHarmfulIntent(journalEntry)

	if journalEntry.harmfulIntent: 
		buildHarmresponse() # reach out to resources 
		return 
	
	return generate_propmt()
	

Generate_Prompt():
	if type == Cathartic:
		response = generateCatharticResponse()
	else:
		response = generateCasualResponse()
	safety_counter = 0
	while(safety_counter<5):
		safe = promptSafetyDetector(journalEntry)
		safety_coutner ++
		if safe:
			break

	if safe:
		return journalEntry 

	# if not safe
	
	reliableQuestionClassifier()
	return journalEntry
	


checkHarmfulIntent(journalEntry):
	if not harm:
		journalEntry.harmfulIntent = False
		return 
	
	journalEntry.harmfulIntent = True



generateCasualResponse():
	# generate empathy 
	# return 


generateCatharticResponse():
	# generate empathy
	# if stage = 5: call cognitive reframing
	# else questionmapper

	# build response using empathy and questions 

	# return 


In [26]:
a = 5

In [29]:
a += 5
a`

20